In [ ]:
# Housekeeping - Jupyter Fehlermeldungen kürzen
# Hinweis: Sorry, bei mybinder.org klappt der Ansatz nicht
import json 
from pprint import pprint
import sys
ipython = get_ipython()
unhide_traceback = None

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

if not unhide_traceback:
    unhide_traceback = ipython.showtraceback

ipython.showtraceback = hide_traceback
# ipython.showtraceback = unhide_traceback



Pythoncamp 2020 Session Martin Borus, Twitter: @mborus

# GLOM

von Mahmoud Hashemi

## "Wenn Du verschachtelste Daten hast, brauchst du Glom!"

https://github.com/mahmoud/glom 





# Hauptfunktionen

- Pfadbasierender Zugriff 
- deklarative Datenumwandlung
- lesbare, ausagekräftige Fehlermeldungen
- mit Debugger!


## Vorbereitung

pip install glom pip install --upgrade -e git+https://github.com/mahmoud/glom#egg=glom

Dokumention und Tutorial auf https://glom.readthedocs.io,
von hier stammen auch die Code-Beispiele der Einführung

# 1. Normales Python

In [ ]:
data = {'a': {'b': {'c': 'd'}}}

In [ ]:
data['a']['b']['c']

In [ ]:
data2 = {'a': {'b': None}}

In [ ]:
data2['a']['b']['c']

# 2. Glom 

In [ ]:
from glom import glom

In [ ]:
glom(data, 'a.b.c')

In [ ]:
glom(data2, 'a.b.c')


In [ ]:
# Mit glom lassen sich Fehler gut abfangen

from glom import GlomError, PathAccessError

try:
    glom(data2, 'a.b.c')
except PathAccessError as e:
    print(e)

In [ ]:
try:
    glom(data2, 'a.b.c')
except AttributeError as e:
    print(e)

In [ ]:
try:
    glom(data2, 'a.b.c')
except GlomError as e:
    print(e)

## Glom geht auch mit Lists

In [ ]:
data = [1, [2, 3, 4] , 3, 4 ,5]

In [ ]:
glom(data, '1.0')

## Glom geht auch mit Objekten !1!!

In [ ]:
class MyClass:
    def __init__(self):
        self._hallo = "hallo!"
        self._welt = "welt!"
       
myvar = MyClass()        

In [ ]:
glom(myvar, '_hallo')

# 3. Target & Spec
- "Target" sind die Daten, (list, dict, object)
- "Spec" ist das gewünschte Ergebnis

In [ ]:
target = {
     'galaxy': {
        'system': {
            'planet': 'jupiter'
         }
    }
}

spec = 'galaxy.system.planet'

glom(target, spec)

In [ ]:
target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}

spec = {
     'names': ('system.planets', ['name']),
     'moons': ('system.planets', ['moons'])
}

glom(target, spec)

In [ ]:
target = {
     'system': {
         'planets': [
            {
                'name': 'earth',
                'moons': [
                    {'name': 'luna'}
                ]
            },
            {
                'name': 'jupiter',
                'moons': [
                    {'name': 'io'},
                    {'name': 'europa'}
                ]
            }
        ]
    }
}

In [ ]:

spec = {
    'planet_names': ('system.planets', ['name']),
    'moon_names': ('system.planets', [('moons', ['name'],  )])
}
pprint(glom(target, spec))

In [ ]:
from glom import Coalesce

target = {
     'system': {
         'planets': [
             {'name': 'earth', 'moons': 1},
             {'name': 'jupiter', 'moons': 69}
         ]
     }
}

spec = {
     'planets': (Coalesce('system.planets', 'system.dwarf_planets'), ['name']),
     'moons': (Coalesce('system.planets', 'system.dwarf_planets'), ['moons'])
}

glom(target, spec)

In [ ]:
target = {
     'system': {
         'dwarf_planets': [
             {'name': 'pluto', 'moons': 5},
             {'name': 'ceres', 'moons': 0},
         ]
     }
 }
glom(target, spec)

In [ ]:
target = {
     'system': {
         'planets': [
             {'name': 'earth', 'moons': 1},
             {'name': 'jupiter', 'moons': 69}
         ]
     }
}

glom(target, {'moon_count': ('system.planets', ['moons'], sum)})
# glom(target, {'moon_count': ('system.planets', ['moons'], lambda x: sum(x))})

In [ ]:
class MySubClass:
    def __init__(self):
        self._hey = "Hey!"

class MyClass:
    def __init__(self):
        self._hallo = "hallo!"
        self._welt = "welt!"
        self._heylist = [MySubClass()] * 6

        
myvar = MyClass()  

In [ ]:
from glom import Iter
spec = {'hallo': '_hallo', 'welt': '_welt', 'heylist': ('_heylist', ['_hey'])}

glom(myvar, spec)

# 4. Daten eindampfen - Flatten und Merge

In [ ]:
from glom import Flatten, Merge

In [ ]:
data = [[1,2], [3], [4], [], [5]]

In [ ]:
glom(data, Flatten())

In [ ]:
data = [{'hallo': 'welt'}, {'hello': 'world'}]

In [ ]:
glom(data, Merge())

# 5. Tutorial: Objekte -> Ausgaben

In [ ]:
from glom.tutorial import * 

In [ ]:
contact = Contact('Julian',
          emails=[Email(email='jlahey@svtp.info')],
                  location='Canada')

In [ ]:
contact.save()

In [ ]:
contact.primary_email

In [ ]:
contact.add_date

In [ ]:
contact.id

In [ ]:
len(Contact.objects.all())

In [ ]:
json.dumps(Contact.objects.all())

In [ ]:
target = Contact.objects.all()

In [ ]:
target[0].add_date

In [ ]:
spec = {'results': [{'id': 'id',
                      'name': 'name',
                      'add_date': ('add_date', str),
                      'emails': ('emails', [{'id': 'id',
                                            'email': 'email',
                                            'type': 'email_type'}]),
                      'primary_email': Coalesce('primary_email.email', default=None),
                      'pref_name': Coalesce('pref_name', 'name', skip='', default=''),
                      'detail': Coalesce('company',
                                         'location',
                                         ('add_date.year', str),
                                         skip='', default='')}]}

In [ ]:
resp = glom(target, spec)

In [ ]:
print(json.dumps(resp, indent=2, sort_keys=True))

# 6. "T" - das Stunt-Double

In [ ]:
from glom import T, Flatten
# Antwort 1. Zeile
glom(resp, ('results', T[0]))

In [ ]:
# Antwort: Alle Emails
from glom import Flatten

glom(resp, ('results', ['emails'],
            Flatten(),
            ['email'],
           )
    )


# 7. Loop mit Iter

In [ ]:
from glom import glom, Iter

target = ['Brummbär',
          'Pimpel',
          'Happy',
          'Chef',
          'Hatschi',
          'Schlafmütz',
          'Seppel']

In [ ]:
# Iter liefert erst einmal einen Generator
spec = Iter()
glom(target, spec)

In [ ]:
# Generator in Liste verwandeln
spec = Iter().all()
glom(target, spec)

In [ ]:
# Gruppieren in Chunks
spec = Iter().chunked(2, fill='Schneewittchen').all()
glom(target, spec)

In [ ]:
spec = Iter().filter(lambda x: x != 'Chef').all()
glom(target, spec)

In [ ]:
# Maximale Größe der Rückgabe
spec = Iter().limit(3).all()
glom(target, spec)

In [ ]:
# Zwerge 2 bis 4 in der 0-basierten Liste finden
spec = Iter().slice(1, 4).all()
glom(target, spec)

In [ ]:
# Nur ein Zwerg mit dem gleichen Anfangsbuchstaben erlaubt
spec = Iter().unique(T[0]).all()
glom(target, spec)

In [ ]:
# Nehmen, nur bis der Chef kommt
spec = Iter().takewhile(lambda x: x != 'Chef').all()
glom(target, spec)

In [ ]:
# Der erste Zwerg, der nach dem Chef kommt...
spec = Iter().dropwhile(lambda x: x != 'Chef').slice(1, 2).first()
glom(target, spec)

In [ ]:
# Gruppentrennung
spec = Iter().split('Chef').all() 
glom(target, spec)

In [ ]:
# Funktion anwenden
spec = Iter().map(lambda x:x.lower()).all()
glom(target, spec)


In [ ]:
# oder das Stunt-Double verwenden
spec = Iter().map(T.upper()).all()
glom(target, spec)

# 8. Literal - Feste Werte vergeben

In [ ]:
from glom import Literal
spec = Iter({'Name': T, 'Grösse': Literal('Zwerg')}).all()

In [ ]:
glom(target, spec)

# 9. Data Driven

Wenn der Schlüssel aus dem Dictionary Daten enthält

In [ ]:
from glom import glom, T, Merge, Iter, Coalesce

target = {
    "pluto": {"moons": 6, "population": None},
    "venus": {"population": {"aliens": 5}},
    "earth": {"moons": 1, "population": {"humans": 7_700_000_000, "aliens": 1}},
}

spec = {
    "moons": (
           T.items(),
           Iter({T[0]: (T[1], Coalesce("moons", default=0))}),
           Merge()
    )
}
        
glom(target, spec)        

# 10. Werte hinzufügen & Löschen

In [ ]:
data = {'moons': {'pluto': 6, 'venus': 0, 'earth': 1}}

In [ ]:
from glom import Assign, Delete
spec = Assign('moons.saturn', 7)

In [ ]:
glom(data, spec)

In [ ]:
spec = Delete('moons.earth')

In [ ]:
glom(data, spec)

In [ ]:
spec = Delete('moons.mars', ignore_missing=False)
glom(data, spec)

# 11. Scope

Mit Scope weitere Daten übergeben, die sonst nicht sichtbar wären

In [ ]:
from glom import S, glom, Assign, Spec

target = {'date': '2020-04-01',
 'location': 'A',
 'items': [
     {'name': 'A', 'id': 'A1'},
     {'name': 'B', 'id': 'B1'},
     {'name': 'C', 'id': 'C1'}
]}

spec = ('items', 
        [Assign('date', Spec(S['date']))], 
        [Assign('location', Spec(S['location']))]
       )

glom(target, spec, scope=target)

# 12. XML

In [ ]:
from glom import Ref

In [ ]:
etree2dicts = Ref('ElementTree',
    {"tag": "tag", 
     "text": "text", 
     "attrib": "attrib", 
     "children": (iter, [Ref('ElementTree')])})

In [ ]:
html_text = """<html>
  <head>
    <title>the title</title>
  </head>
  <body id="the-body">
    <p>A paragraph</p>
  </body>
</html>"""

In [ ]:
from xml.etree import ElementTree
etree = ElementTree.fromstring(html_text)

In [ ]:
glom(etree, etree2dicts)

# 13. Weiteres...

aus Zeitgründen noch nicht erwähnt

- Path ( Eine andere Pfadnotation, wenn Strings nicht gehen)
- Invoke / Call (Funktionen anwenden)
- Check -> CheckError (Daten auf Korrektheit prüfen!)
- Inspect (Debugger)  

# 14. Hands on

Beispiel: Aus der Raumliste http://borus.de/pythoncamp/event.json ein Dictionary erstellen, das den Raumnamen in GROSSBUCHSTABEN als Schlüssel hat und die URL als Wert.

Gewünschtes Ergebnis:

    {
     'BERLIN': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
     'FLIEGENDER ZIRKUS': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm',
     'TOKIO': 'https://bbb01.pythoncamp.online/b/rei-hyz-cgm'
     }

In [ ]:
import requests

In [ ]:
r = requests.get(r'http://borus.de/pythoncamp/event.json')

In [ ]:
r.json()

In [ ]:
spec = ...

In [ ]:
glom(r.json(), spec)